In [30]:
import mysql.connector
from mysql.connector import Error
import pandas as pd

# Sql Connector


### Connection to MySql workbench

In [15]:
def connecting_MySql(host_name,user_name,user_password):
    Connection=None
    try:
        Connection=mysql.connector.connect(host=host_name,user=user_name,passwd=user_password)
        return Connection
    except Error as err:
        print(f"Error: `{err}`")

pw='Your Password' #Enter you password
Host_name="localhost" #Enter your Host Name, example:- localhost
User_name="root"   #Enter your User name, Example:- root

connection=connecting_MySql(Host_name,User_name,pw)

### Creating Database

In [31]:
def create_database(connection,query):
    cursor=connection.cursor()
    try:
        cursor.execute(query)
    except Error as err:
        print(f"Error: `{err}`")

Query="create database Library;"
create_database(connection,Query)
db="Library"

### Login into database

In [17]:
def connecting_database(host_name,db_user_name,db_password,database_name):
    connection=None
    try:
        connection=mysql.connector.connect(
            host=host_name,
            user=db_user_name,
            passwd=db_password,
            database=database_name
        )
        return connection
    except Error as err:
      print(f"Error: `{err}`")
connection=connecting_database(Host_name,User_name,pw,db)

### Connecting table

In [18]:
def executing_query(connection,query):
    cursor=connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
    except Error as err:
        print(f"Error: `{err}`")

def Printing_table(connection,query):
    cursor=connection.cursor()
    ressult=None
    try:
        cursor.execute(query)
        result=cursor.fetchall()
        return result
    except Error as err:
        print(f"Error: `{err}`")

### Creating table for user

In [32]:
user_table='''create table User_table(ID int primary key auto_increment , 
Name varchar(50) not null);'''

user_book='''create table user_book(Book_no int primary key auto_increment,
Book varchar(100) not null, 
author_name varchar(100),
Quantity int, 
Book_taken_date date, 
Library_id int,
Lib_Book_no int,
user_id int not null);'''
connection = connecting_database(Host_name,User_name,pw,db)
executing_query(connection,user_table)
executing_query(connection,user_book)

### creating table for library



In [33]:
library_owner='''create table library_owner(Library_ID int primary key auto_increment, 
Lib_Name varchar(50) not null);'''

library_book='''create table library_book(Book_no int primary key auto_increment,
Book varchar(100) not null,
Author varchar(50),
Quantity int,
Lib_id int,
Lib_name varchar(100));'''

connection = connecting_database(Host_name,User_name,pw,db)
executing_query(connection,library_owner)
executing_query(connection,library_book)

### Trigger

In [34]:
Trigger=f'''
create trigger trigger_update
before update on library_book
for each row
begin
    if new.Quantity<0 then
         set @meess=Concat('Sorry, Book is low or out of stock try other book');
         signal sqlstate '45000'
         set message_text=@meess;
    end if;
end;
'''

connection = connecting_database(Host_name,User_name,pw,db)
executing_query(connection,Trigger)

# Library Class

In [23]:
class library:
    def __init__(self,ID,Name):
        self.ID=ID
        self.Name=Name
        
    def Add_New_Book(self,bookName,AuthorName,Quantity):
        Query=f'''insert into library_book(Book,Author,Quantity,Lib_id,Lib_name) 
        values("{bookName}","{AuthorName}","{Quantity}",{self.ID},"{self.Name}");'''
        connection = connecting_database(Host_name,User_name,pw,db)
        executing_query(connection,Query)
        
    def display(self):   
        Query=f"Select Book_no,Book,Author,Quantity from library_book where (Lib_id={self.ID}) and (Lib_name='{self.Name}');"
        connection = connecting_database(Host_name,User_name,pw,db)
        Resultant=Printing_table(connection,Query)
        Output=[]
        for result in Resultant:
            result = list(result)
            Output.append(result)
        columns = ["Book no","Book Name","Author Name","Quantity"]
        df = pd.DataFrame(Output,columns=columns)
        display(df)
        print("\n")

    def student_taken_book(self):
        Query=f'Select Book_no,Book,author_name,Quantity,Book_taken_date,user_id from user_book where Library_id="{self.ID}";'
        connection = connecting_database(Host_name,User_name,pw,db)
        Resultant=Printing_table(connection,Query)
        Output=[]
        for result in Resultant:
            result = list(result)
            Output.append(result)
        columns = ["Book No","Book Name","Author Name","Quantity","Date Taken","User Id",]
        df = pd.DataFrame(Output,columns=columns)
        display(df)
        print("\n")
    
    def Remove(self,Book_no):
        Query=f"delete from library_book where Book_no='{Book_no}';"
        connection = connecting_database(Host_name,User_name,pw,db)
        executing_query(connection,Query)
        
    def Add_Quantity(self,Quantity,Book_no):
        Query=f"update library_book set Quantity='{Quantity}' where Book_no='{Book_no}';"
        connection = connecting_database(Host_name,User_name,pw,db)
        executing_query(connection,Query)
        
    def change_book_name(self,NewName,Book_no):
        Query=f"update library_book set Book='{NewName}' where Book_no='{Book_no}';"
        connection = connecting_database(Host_name,User_name,pw,db)
        executing_query(connection,Query)
        
    def change_authou_name(self,NewAuthorName,Book_no):
        Query=f"update library_book set Author='{NewAuthorName}' where Book_no='{Book_no}';"
        connection = connecting_database(Host_name,User_name,pw,db)
        executing_query(connection,Query)
        

# User Class

In [24]:
class User:
    def __init__(self,ID,User_Name):
        self.User_Name=User_Name
        self.User_ID=ID
        
    def display_all_book(self):
        Query=f'''select lb.Book_no,lb.Book,lb.Author,lb.Quantity,lb.Lib_id,lb.Lib_name from 
        library_book lb join library_owner lo on lb.Lib_id=lo.Library_ID;'''
        connection = connecting_database(Host_name,User_name,pw,db)
        Resultant=Printing_table(connection,Query)
        Output=[]
        for result in Resultant:
            result = list(result)
            Output.append(result)
        columns = ["Book No","Book Name","Author Name","Quantity","Lib Id","Lib Name"]
        df = pd.DataFrame(Output,columns=columns)
        display(df)        
        print("\n")
  
    def Take_book(self,Book_name,Author_name,Quantity,Lib_id,Book_no):
        Query=f"select Book_no from library_book;"
        connection = connecting_database(Host_name,User_name,pw,db)
        Resultant=Printing_table(connection,Query)
        Output=[]
        for result in Resultant:
            result = list(result)
            Output.append(result)
        Validation=False
        for i in Output:
            if Book_no==i[0]:
                Validation=True
                break
        if (Validation):
            Query = f'''INSERT INTO user_book (Book, author_name, Quantity, Book_taken_date, 
            Library_id,Lib_Book_no, user_id) VALUES ('{Book_name}', '{Author_name}', {Quantity}, CURDATE(), {Lib_id},{Book_no},{self.User_ID});'''
            connection = connecting_database(Host_name,User_name,pw,db)
            update_lib_quantity=f"update library_book set Quantity=Quantity-{Quantity} where Book_no='{Book_no}';"
            executing_query(connection,Query)
            executing_query(connection,update_lib_quantity)
            
        else:
            print("Invalid Book")
    
    def display_Borrowed_books(self):
        Query=f'Select Book_no,Book,author_name,Quantity,Book_taken_date,Library_id,Lib_Book_no from user_book where user_id="{self.User_ID}";'
        connection = connecting_database(Host_name,User_name,pw,db)
        Resultant=Printing_table(connection,Query)
        Output=[]
        for result in Resultant:
            result = list(result)
            Output.append(result)
        columns = ["Book no","Book Name","Author Name","Quantity","Date Taken","Lib Id","Library Book No"]
        df = pd.DataFrame(Output,columns=columns)
        display(df)
        print("\n")
        
    def Give_back_book(self,Quantity,Book_no,Lib_Book_no):
        Query=f"select Book_no from library_book;"
        connection = connecting_database(Host_name,User_name,pw,db)
        Resultant=Printing_table(connection,Query)
        Output=[]
        for result in Resultant:
            result = list(result)
            Output.append(result)
        Validation=False
        for i in Output:
            if Lib_Book_no==i[0]:
                Validation=True
                break
                
        if Validation:
            Query=f"update library_book set Quantity=Quantity+{Quantity} where Book_no='{Lib_Book_no}';"
            Remove=f"update user_book set Quantity=Quantity-{Quantity} where Book_no='{Book_no}';"
            Checking=f"select Quantity from user_book where Book_no={Book_no} and user_id={self.User_ID};"
            connection = connecting_database(Host_name,User_name,pw,db)
            executing_query(connection,Query)
            executing_query(connection,Remove)
            Resultant=Printing_table(connection,Checking)
            Output=[]
            for result in Resultant:
                result = list(result)
                Output.append(result)
            for i in Output:
                if (i[0]<=0):
                    Remove_book=f"delete from user_book where Book_no={Book_no};"
                    connection = connecting_database(Host_name,User_name,pw,db)
                    executing_query(connection,Remove_book)
                    break
        else:
            print("Invalid book number")


# User verfication Class


In [25]:
class User_validation:
    def __init__(self,ID,Name):
        self.ID=ID
        self.Name=Name
        
    def library_user(self):
        Query="select * from library_owner;"
        connection = connecting_database(Host_name,User_name,pw,db)
        Resultant=Printing_table(connection,Query)
        Output=[]
        for result in Resultant:
            result = list(result)
            Output.append(result)
        return [self.ID, self.Name] in Output
            
    def User_verification(self):
        Query="select * from User_table;"
        connection = connecting_database(Host_name,User_name,pw,db)
        Resultant=Printing_table(connection,Query)
        Output=[]
        for result in Resultant:
            result = list(result)
            Output.append(result)
        return [self.ID, self.Name] in Output

# Main

In [35]:
print("\n\n")
print("Hello.. Welcome to Library Management")
print("Which Role defnes you :)=")
print("1. Library")
print("2. User")
Role=int(input("Enter the option number="))

if(Role==1):
    Existing=input("Want to register as New library(Yes/No)=")

    if(Existing.lower()=='yes'):
        Name=input("Enter your library name=").title()
        Query_insert=f"insert into library_owner(Lib_Name) values ('{Name}');"
        Query=f"select Library_ID from library_owner where Lib_Name = '{Name}';"
        connection = connecting_database(Host_name,User_name,pw,db)
        executing_query(connection,Query_insert)
        Resultant=Printing_table(connection,Query)
        for result in Resultant:
            print("Your Id=",result[0])
            
    elif(Existing.lower()=='no'):
        ID=int(input("\tEnter your Library ID="))
        Name=input("\tEnter your Library name=").title()
        User_checker=User_validation(ID,Name)
        Checking=User_checker.library_user()
        Library=library(ID,Name)
        
        if(Checking):
            while(True):   
                print("\n\t1. Display all books")
                print("\t2. Update the book")
                print("\t3. Display the book taken by user")
                print("\t4. To exit")
                Operation=int(input("\tWhat operation you want to perform="))
            
                
                if(Operation==1):
                    Library.display()
                            
                elif(Operation==2):
                    print("\n\t\t1. Add New Book")
                    print("\t\t2. Remove the book")
                    print("\t\t3. Change the book name")
                    print("\t\t4. Change the author name")
                    print("\t\t5. Add the quantity")
                    Operation=int(input("\t\tEnter the number which u want to perform"))
            
                    if(Operation==1):
                        Library.display()
                        BookName=input("Enter the book name=").title()
                        AuthorName=input("Enter the Author name=").title()
                        Quantity=int(input("Enter the number of quantity="))
                        Library.Add_New_Book(BookName,AuthorName,Quantity)
                        
                    if(Operation==2):
                        Library.display()
                        BookNumber=int(input("Enter the valid book number="))
                        Library.Remove(BookNumber)
                                
                    elif(Operation==3):
                        Library.display()
                        BookNumber=int(input("Enter the valid book number="))
                        NewName=input("Enter the new book name=").title()
                        Library.change_book_name(NewName,BookNumber)
                                
                    elif(Operation==4):
                        Library.display()
                        BookNumber=int(input("Enter the valid book number="))
                        NewAuthor=input("Enter the new author name=").title()
                        Library.change_authou_name(NewAuthor,BookNumber)
                                
                    elif(Operation==5):
                        Library.display()
                        BookNumber=int(input("Enter the valid book number="))
                        NewQuantity=int(input("Enter the number of quantity to be updated="))
                        Library.Add_Quantity(NewQuantity,BookNumber)
                                
                elif(Operation==3):
                    Library.student_taken_book()
                    
                elif(Operation==4):
                    print("Thank You For Using Our Service")
                    break
                    
                else:
                    print("Invalid input")
            else:
                 print("Check you login credentials. OR Create new ID")

elif(Role==2):
        Existing=input("Want to register as new User(Yes/No)=")
        if(Existing.lower()=='yes'):
            Name=input("Enter your  name=").title()
            Query_insert=f"insert into user_table(Name) values ('{Name}');"
            Query=f"select ID from user_table where Name = '{Name}';"
            connection = connecting_database(Host_name,User_name,pw,db)
            executing_query(connection,Query_insert)
            Resultant=Printing_table(connection,Query)
            for result in Resultant:
                print("Your Id=",result[0])
                
        elif(Existing.lower()=='no'):
            ID=int(input("\tEnter your ID="))
            Name=input("\tEnter your name=").title()
            User_checker=User_validation(ID,Name)
            Checking=User_checker.User_verification()
            User_Student=User(ID,Name)
            if (Checking):
                while(True):
                    print("\t\t1. Display All books present in Library")
                    print("\t\t2. Display taken books from Library")
                    print("\t\t3. Take Books")
                    print("\t\t4. Give back book")
                    print("\t\t5. to exit")
                    Operation=int(input("\tEnter the number to be performed="))
                    User_Student=User(ID,Name)
                    
                    if(Operation==1):
                        User_Student.display_all_book()
                        
                    elif(Operation==2):
                        User_Student.display_Borrowed_books()
                        
                    elif(Operation==3):
                        User_Student.display_all_book()
                        Book_name = input("Enter the book name: ").title()
                        author_name = input("Enter the author name: ").title()
                        quantity = int(input("Enter the quantity: "))
                        lib_id = int(input("Enter the library ID: "))
                        Book_no=int(input("Enter the book number="))
                        User_Student.Take_book(Book_name,author_name,quantity,lib_id,Book_no)
                        
                    elif(Operation==4):
                        User_Student.display_Borrowed_books()
                        Quantity=int(input("Enter the Quantity number="))
                        Book_no=int(input("Enter the book number="))
                        Lib_Book_no=int(input("Enter the Library book number="))
                        User_Student.Give_back_book(Quantity,Book_no,Lib_Book_no)
                        
                    elif(Operation==5):
                        print("Thank You For Using Our Service")
                        break
                        
                    else:
                        print("Wrong number")
            else:
                print("Check you login credentials. OR Create new ID")